In [59]:
%pip install google-api-python-client
%pip install google-auth google-auth-oauthlib google-auth-httplib2
%pip install pickle sastrawi textblob
%pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


Note: you may need to restart the kernel to use updated packages.


In [60]:
#library yang digunakan, jika dirasa kurang penting dapat dihapus
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import re
from nltk import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from textblob import TextBlob
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
from googleapiclient.discovery import build
import pandas as pd
import re

def extract_video_id(url):
    match = re.search(r"(?:v=|\/)([a-zA-Z0-9_-]{11})", url)
    return match.group(1) if match else None

def get_comments(api_key, url):
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    single_video_id = extract_video_id(url) if "youtube.com" in url else url
    if not single_video_id:
        print("Invalid video URL or ID")
        return

    video_list = youtube.videos().list(part="snippet", id=single_video_id).execute()
    if not video_list["items"]:
        print("Video not found or no access.")
        return

    nextPageToken_comments = None
    commentsone = []

    while True:
        pl_request_comment = youtube.commentThreads().list(
            part=["snippet", "replies"],
            videoId=single_video_id,
            maxResults=50,
            pageToken=nextPageToken_comments
        )
        pl_response_comment = pl_request_comment.execute()

        for i in pl_response_comment["items"]:
            vid_comments = i["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
            comm_author = i["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
            comm_author_id = i["snippet"]["topLevelComment"]["snippet"]["authorChannelId"]["value"]
            comm_date = i["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
            comm_likes = i["snippet"]["topLevelComment"]["snippet"]["likeCount"]

            # Tambahan untuk mendapatkan reply jika ada
            new_var = i.get("replies", "0")

            commentsone.append({
                "comm_date": comm_date,
                "author": comm_author,
                "author_id": comm_author_id,
                "likes": comm_likes,
                "comment": vid_comments,
                "replies": new_var,
                "video_id": single_video_id
            })

        nextPageToken_comments = pl_response_comment.get("nextPageToken")
        if not nextPageToken_comments:
            break

    # Simpan data komentar dengan replies ke CSV
    df = pd.DataFrame.from_dict(commentsone)
    df.to_csv("dataset.csv", index=False)

    # Cetak 10 komentar pertama untuk verifikasi
    for i in commentsone[:10]:
        print(i["comment"], i["replies"])

# Input API Key dan URL
api_key = input("API KEY: ")
url = input("VIDEOURL: ")
get_comments(api_key, url)


Video not found or no access.


In [62]:
import nltk
import pandas as pd

# Membaca file CSV yang berisi dataset komentar
data = pd.read_csv("./dataset.csv")

# Menghapus baris yang memiliki nilai kosong (NaN) dari dataset
data = data.dropna()

# Mencetak 5 baris pertama dari dataset yang telah dibersihkan
print(data.head())


              comm_date             author                 author_id  likes  \
0  2024-08-29T04:47:30Z   @DansReddlesCorp  UCHaQYPcOclOgXm6RCIs2W6Q      0   
1  2024-08-28T11:49:02Z        @yoenisu484  UC4KpgwdaoWOev0pENvRLv7g      0   
2  2024-08-27T18:15:04Z     @Wisatachanell  UChNzLkzQxjdEpepN_yT_mRA      0   
3  2024-08-27T09:39:59Z   @RediManlaia-t7g  UC6XZ2EKwZfwHVui_G1ZYSUQ      0   
4  2024-08-27T04:25:30Z  @satriawibawa9079  UCZ8qjsVcd9Zt_PBPM-lkeCA      0   

                                             comment replies     video_id  
0  Tolak RUU pilkada mati matian demo,, sekali pa...       0  rONUSWIPhbA  
1                      Darurat ????? Ga usah lebayyy       0  rONUSWIPhbA  
2  Mending mengesahkan revisi UUD perlindungan PR...       0  rONUSWIPhbA  
3  Ricuh dan perusakan  itu akibat dari kurangnya...       0  rONUSWIPhbA  
4  UU koruptor coba demo mahasiswa.. \nDiem aja 😂...       0  rONUSWIPhbA  


In [63]:
data_nw = data.drop(['comm_date',"author", 'author_id',"likes",'video_id'], axis=1 )
data_nw

,comment,replies
0,"Tolak RUU pilkada mati matian demo,, sekali pa...",0
1,Darurat ????? Ga usah lebayyy,0
2,Mending mengesahkan revisi UUD perlindungan PR...,0
3,Ricuh dan perusakan itu akibat dari kurangnya...,0
4,UU koruptor coba demo mahasiswa.. \nDiem aja 😂...,0
...,...,...
67,Dimana ada praktek dinasti pasti rakyat tersak...,"{'comments': [{'kind': 'youtube#comment', 'eta..."
68,Semoga cepat sembuh di negeri ini..,0
69,Disini menegaskan bahwa tetap PDIP partai yg i...,"{'comments': [{'kind': 'youtube#comment', 'eta..."
70,pertama,0


In [64]:
data_nw.to_csv("C:/Users/user/Documents/GitHub/SentimentAnalysis-Bengkod/datasets/dataset_drop.csv") #Fungsinya untuk menyimpan hasil drop

In [65]:
data_baru = pd.read_csv("C:/Users/user/Documents/GitHub/SentimentAnalysis-Bengkod/datasets/dataset_drop.csv", index_col=0)
data_baru.head()

,comment,replies
0,"Tolak RUU pilkada mati matian demo,, sekali pa...",0
1,Darurat ????? Ga usah lebayyy,0
2,Mending mengesahkan revisi UUD perlindungan PR...,0
3,Ricuh dan perusakan itu akibat dari kurangnya...,0
4,UU koruptor coba demo mahasiswa.. \nDiem aja 😂...,0


In [66]:
def caseFolding(comment):
          comment = comment.lower()
          comment = comment.strip(" ")
          comment = re.sub(r'[?|$|.|!]',r'', comment)
          comment = re.sub(r'[^a-zA-Z0-9 ]',r'', comment)
          return comment

data_baru['comment'] = data_baru['comment'].apply(caseFolding)

In [68]:
data_baru.to_csv("C:/Users/user/Documents/GitHub/SentimentAnalysis-Bengkod/datasets/dataset_bersih.csv")

In [69]:
print(data_baru.columns)

Index(['comment', 'replies'], dtype='object')


In [72]:
import pandas as pd

data = pd.read_csv('C:/Users/user/Documents/GitHub/SentimentAnalysis-Bengkod/datasets/dataset_bersih.csv')
X = data['comment']
y = data['sentiment']

In [73]:
# Ekstraksi fitur menggunakan TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000)
X = tfidf.fit_transform(data['comment']).toarray()
y = data['sentiment']

In [74]:
# Membagi data menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
# Melatih model KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [76]:
# Memprediksi data testing
y_pred = knn.predict(X_test)

In [77]:
# Evaluasi model
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {acc * 100:.2f}%")

Confusion Matrix:
[[9 1 0]
 [1 0 0]
 [3 1 0]]
Accuracy: 60.00%


In [78]:
# Fungsi untuk memprediksi sentimen dari komentar baru
def predict_sentiment(comment):
    comment = caseFolding(comment)
    comment_tfidf = tfidf.transform([comment]).toarray()
    prediction = knn.predict(comment_tfidf)
    return prediction[0]

In [79]:
# Input komentar baru untuk prediksi sentimen
new_comment = input("Masukkan komentar: ")
sentiment = predict_sentiment(new_comment)
print(f"Komentar: {new_comment}")
print(f"Sentimen komentar tersebut adalah: {sentiment}")

Komentar: anies bagus performa nya
Sentimen komentar tersebut adalah: netral


### **KNN**

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Download stopwords jika belum terunduh
nltk.download('stopwords')

# Fungsi untuk preprocessing teks
def preprocessing(comment):
    # Mengubah teks menjadi huruf kecil
    comment = comment.lower()
    # Menghapus angka
    comment = re.sub(r'\d+', '', comment)
    # Menghapus tanda baca
    comment = re.sub(r'[^\w\s]', '', comment)
    # Menghapus spasi di awal dan akhir
    comment = comment.strip()
    # Tokenisasi
    tokens = nltk.word_tokenize(comment)
    # Menghapus stopwords
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Stemming menggunakan Sastrawi
    stemmer = StemmerFactory().create_stemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    # Menggabungkan kembali token yang telah diproses
    return ' '.join(stemmed_tokens)

# Terapkan preprocessing pada kolom 'comment'
data['comment'] = data['comment'].apply(preprocessing)

# Simpan dataset yang telah diproses
data.to_csv("dataset_preprocessed.csv", index=False)

# Tampilkan 5 baris pertama dari dataset yang telah diproses
print(data.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\user/nltk_data'
    - 'c:\\Users\\user\\Documents\\GitHub\\SentimentAnalysis-Bengkod\\myenv\\nltk_data'
    - 'c:\\Users\\user\\Documents\\GitHub\\SentimentAnalysis-Bengkod\\myenv\\share\\nltk_data'
    - 'c:\\Users\\user\\Documents\\GitHub\\SentimentAnalysis-Bengkod\\myenv\\lib\\nltk_data'
    - 'C:\\Users\\user\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/Users/user/AppData/Roaming/nltk_data'
**********************************************************************
